In [ ]:
import redis
import time
import pickle
import cv2

In [ ]:
stream_key = 'Inpainting'
group = "worker"
ack_stream_key = 'finish_ack'

conn = redis.Redis()
conn.ping()

In [ ]:
for i in range(0,10):
    conn.xadd( stream_key, { 'ts': time.time(), 'v': i } )

In [ ]:
t = conn.xadd('skey',{'ts':time.time()})

In [ ]:
img_path = r"C:\Users\xyfuture\Pictures\origin.png"
mask_path = r"C:\Users\xyfuture\Pictures\mask.png"

img = cv2.imread(img_path)
mask = cv2.imread(mask_path)

first_img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
first_mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)

In [ ]:
conn.xadd('Inpainting',{'image':pickle.dumps(first_img),'mask':pickle.dumps(first_mask)})

In [ ]:
conn.xgroup_create( name='Inpainting', groupname='worker', id=0)

In [ ]:
conn.xadd('Inpainting',{'image':123,'mask':456})

In [ ]:
l = conn.xread(streams={'finish_ack':0},block=1)

In [ ]:
tmp = pickle.loads(l[0][1][1][1][b'results'])

In [27]:
print(type(tmp))

<class 'numpy.ndarray'>


In [28]:
import numpy as np
out = tmp.astype(np.uint8)
# out = cv2.cvtColor(out,cv2.COLOR_BGR2RGB)
cv2.imwrite(f'out.jpg', out)

True